# ODE Integrators I: Explicit Methods

Ordinary differential equations form the mathematical foundation of
physics.
They describe how systems evolve in time, e.g., Newton's second law of
motion is
\begin{align}
  f = m a = m \frac{d^2x}{dt^2},
\end{align}
which is a second-order ODE relating force to acceleration.
More generally, an ODE relates a function of time to its time
derivatives.

While ODEs capture the essence of dynamics, analytic solutions are
possible only in special cases.
Most real systems involve nonlinear forces, damping, or external
driving terms that make exact solutions impossible.
Numerical integration allows us to approximate trajectories step by
step and study systems far beyond what can be solved by hand.

## Problem Definition and Types of ODEs

Consider two forms of first-order ODEs:

1. Time-dependent forcing only:
   \begin{align}
     \frac{dx}{dt} = f(t).
   \end{align}
   Here, the derivative depends only on time.
   Solutions are obtained by direct integration:
   \begin{align}
     x(t) = x(t_0) + \int_{t_0}^{t} f(t') \, dt'.
   \end{align}

2. State- and time-dependent dynamics:
   \begin{align}
     \frac{dx}{dt} = f(x, t).
   \end{align}
   Now, the derivative depends on the state $x$ itself.
   The function we want to solve for also appears inside the RHS,
   making direct integration impossible.
   We cannot evaluate the integral without already knowing $x(t)$ at
   intermediate points.

The first type reduces to standard numerical quadrature (trapezoidal
rule, Simpson's rule, etc.), which we studied earlier.
The second case, nonlinear dependence on both $x$ and $t$, is the
typical situation in physics.
Examples include planetary orbits, nonlinear oscillators, chaotic
systems, and interacting biological populations.
In such cases:
* Direct integration is often not feasible, the problem must be solved
  as an initial value problem (IVP).
* Analytic solutions are often unknown or intractable.
* Numerical methods approximate the solution incrementally, using
  small time steps to trace the system's evolution.

By discretizing time and advancing step-by-step, we can model the
behavior of even the most complex systems.
This is the core idea behind numerical ODE integrators.

## Euler's Method

Euler's method is the simplest techniques for numerically solving
ordinary differential equations.
This method provides an easy way to approximate the solution of an IVP
by advancing one small step at a time.

We can apply Euler's method to an ODE of the form:
\begin{align}
  \frac{dx}{dt} = f(x, t), \quad x(t_0) = x_0
\end{align}
where $x_0$ is the initial value of $x$ at time $t = t_0$.
However, as we will see below, it is usually not recommanded in
pratical calculations because of its stability properties.

### Forward (Explicit) Euler Method

There are three simple ways to derive Euler's method.
The easiest way is simply hold $x$ fixed in $f(x, t)$ and apply the
left Reimann sum.
The left Reimann sum is first order in step size by approximating
$f(x, t)$ as a constant.
In this sense, holding $x$ is somewhat "self-consistent" in terms of
approximation order.

We then recall the definition of a deriviative:
\begin{align}
  f(x, t) = \frac{dx}{dt} = \lim_{\Delta t\rightarrow 0}\frac{x(t + \Delta t) - x(t)}{\Delta t}.
\end{align}
If we simply remove the limit and keep the "finite difference" part,
then it is trivial to show
\begin{align}
  x(t + \Delta t) &\approx x(t) + f(x(t), t)\Delta t.
\end{align}
Which is nothing but again the forward Euler method.
While very intuitive, the above two derivations do not formally show
the order of the Euler method.

We may also consider a numerical approximation to the solution of an
ODE.
We approximate the solution at time $t_{n+1} = t_n + \Delta t$ by
using the Taylor expansion:
\begin{align}
  x(t_{n+1}) = x(t_n) + f(x(t_n), t_n) \Delta t + \mathcal{O}(\Delta t^2)
\end{align}
Neglecting the higher-order terms in the expansion, we obtain once
again the Forward Euler Method:
\begin{align}
  x_{n+1} = x_n + f(x_n, t_n) \Delta t
\end{align}
It is thus a step-by-step approach that proceeds by evaluating $f(x,
t)$ at each time point and then advancing to the next point.
It is an explicit method in 1st order.

Let's solve the simple differential equation:
\begin{align}
  \frac{dx}{dt} = \lambda x(t)
\end{align}

This equation has solution
\begin{align}
  x(t) = \exp[\lambda(t-t_0)]
\end{align}

If we choose $\lambda = 1$ and $x(0) = 1$, the solutoin reduces to
$x(t) = \exp(t)$.

In [ ]:
# Let's visualize the solution:

import numpy as np
from matplotlib import pyplot as plt

T = np.linspace(0, 2, 201)
X = np.exp(T)

plt.plot(T, X)

In [ ]:
# Let's implement Euler's method, with history

def forwardEuler(f, x, t, dt, n):
    T = [t]
    X = [x]
    for i in range(n):
        t += dt
        x += f(x) * dt
        T.append(t)
        X.append(x)
    return T, X

In [ ]:
# Let's test Euler's method

f = lambda x: x

Tf, Xf = forwardEuler(f, 1, 0, 0.1, 20)

plt.plot(T, X)
plt.plot(Tf, Xf, 'o-')
plt.xlabel('t')
plt.ylabel('x')

In [ ]:
# As always, we can study the convergence of the numerical method

def errorf(N=200):
    Tf, Xf = forwardEuler(f, 1, 0, 2/N, N)
    X = np.exp(Tf)
    return np.max(abs(Xf - X))

N  = np.array([64, 128, 256, 512, 1024])
Ef = np.array([errorf(n) for n in N])

plt.loglog(N, 16/N, label='1/N')
plt.loglog(N, Ef, 'o:', label='Forward Euler')
plt.xlabel('N')
plt.ylabel(r'$\text{err} = \max|x_\text{numeric} - x|$')
plt.legend()